# Inference

In [6]:
from transformers import(
    EncoderDecoderModel,
    PreTrainedTokenizerFast,
    # XLMRobertaTokenizerFast,
    BertJapaneseTokenizer,
    BertTokenizerFast,
)

import torch
import csv

In [7]:
encoder_model_name = "cl-tohoku/bert-base-japanese-v2"
decoder_model_name = "skt/kogpt2-base-v2"

src_tokenizer = BertJapaneseTokenizer.from_pretrained(encoder_model_name)
trg_tokenizer = PreTrainedTokenizerFast.from_pretrained(decoder_model_name)
model = EncoderDecoderModel.from_pretrained("./dump/best_model")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [12]:
text = "ギルガメッシュ討伐戦"
# text = "ギルガメッシュ討伐戦に行ってきます。一緒に行きましょうか？"

def translate(text_src):
    embeddings = src_tokenizer(text_src, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
    embeddings = {k: v for k, v in embeddings.items()}
    output = model.generate(**embeddings)[0, 1:-1]
    text_trg = trg_tokenizer.decode(output.cpu())
    return text_trg

print(translate(text))

'길가메시 토벌전'

# Evaluation

In [4]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
smoothie = SmoothingFunction().method4

In [5]:
from tqdm import tqdm
from statistics import mean

bleu = []
f1 = []

DATA_ROOT = './output'
FILE_JP_KO_TEST = 'ja_ko_test.csv'
FILE_FFAC_TEST = 'ffac_test.csv'

with torch.no_grad(), open(f'{DATA_ROOT}/{FILE_FFAC_TEST}', 'r') as fd:
# with torch.no_grad(), open(f'{DATA_ROOT}/{FILE_JP_KO_TEST}', 'r') as fd:
    reader = csv.reader(fd)
    next(reader)
    datas = [row for row in reader]    

    for data in tqdm(datas, "Testing"):
        input, label = data
        embeddings = src_tokenizer(input, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
        embeddings = {k: v for k, v in embeddings.items()}
        with torch.no_grad():
            output = model.generate(**embeddings)[0, 1:-1]
        preds = trg_tokenizer.decode(output.cpu())

        bleu.append(sentence_bleu([label.split()], preds.split(), weights=[1,0,0,0], smoothing_function=smoothie))

print(f"Bleu score: {mean(bleu)}")

Testing:   0%|          | 0/267 [00:00<?, ?it/s]/home/tikim/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Testing: 100%|██████████| 267/267 [01:01<00:00,  4.34it/s]

Bleu score: 0.9619225967540574
